In [1]:
import os
import numpy as np
import csv
import soundfile as sf
from scipy.signal import butter, sosfilt
import subprocess

def convert_ogg_to_wav(input_folder):
    output_folder = input_folder.rstrip("/\\") + "_wav"
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".ogg"):
            in_path = os.path.join(input_folder, filename)
            out_path = os.path.join(output_folder, filename.replace(".ogg", ".wav"))
            try:
                data, sr = sf.read(in_path)
                sf.write(out_path, data, sr)  # WAV로만 저장
                print(f"🎧 ogg → wav 성공: {filename}")
            except Exception as e:
                print(f"❌ 변환 실패 ({filename}): {e}")
    return output_folder


def bandpass_filter(data, sr, lowcut, highcut, order=5):
    sos = butter(order, [lowcut, highcut], btype='band', fs=sr, output='sos')
    return sosfilt(sos, data)

def update_filter_log(csv_path, species_name, lowcut, highcut, note=""):
    # 기존 로그 읽기
    rows = []
    found = False
    if os.path.isfile(csv_path):
        with open(csv_path, mode='r', encoding='utf-8', newline='') as f:
            reader = csv.DictReader(f)
            for row in reader:
                if row['species'] == species_name:
                    # 기존 행 덮어쓰기
                    row['lowcut'] = str(lowcut)
                    row['highcut'] = str(highcut)
                    row['note'] = note
                    found = True
                rows.append(row)

    # 없던 종이면 새로 추가
    if not found:
        rows.append({
            'species': species_name,
            'lowcut': str(lowcut),
            'highcut': str(highcut),
            'note': note
        })

    # CSV 파일 덮어쓰기
    with open(csv_path, mode='w', newline='', encoding='utf-8') as f:
        fieldnames = ['species', 'lowcut', 'highcut', 'note']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)
        print(f"📄 로그 {'업데이트' if found else '추가'}: {species_name} ({lowcut}-{highcut} Hz)")


# 🎛 STEP 3. wav 필터링
def filter_wav(input_folder, lowcut=1000.0, highcut=8000.0, order=5, note="", log_path="species_filter_log.csv"):
    output_folder = input_folder.rstrip("/\\") + "_filtered"
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".wav"):
            in_path = os.path.join(input_folder, filename)
            out_path = os.path.join(output_folder, filename)

            try:
                data, sr = sf.read(in_path)
                if len(data.shape) == 2:
                    data = data.mean(axis=1)

                filtered_data = bandpass_filter(data, sr, lowcut, highcut, order)
                
                sf.write(out_path, filtered_data, sr)  # WAV로 저장
                print(f"✅ 필터링 완료: {filename}")

            except Exception as e:
                print(f"❌ 필터링 실패 ({filename}): {e}")

    species_name = os.path.basename(input_folder.rstrip("/\\"))
    update_filter_log(log_path, species_name, lowcut, highcut, note)
    
    return output_folder

def convert_wav_to_ogg_ffmpeg(input_folder):
    output_folder = input_folder.rstrip("/\\") + "_ogg"
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(".wav"):
            wav_path = os.path.join(input_folder, filename)
            ogg_path = os.path.join(output_folder, filename.replace(".wav", ".ogg"))

            ffmpeg_path = 'D:/Download/ffmpeg-7.1.1-full_build/bin/ffmpeg.exe'

            command = [ffmpeg_path, "-y", "-i", wav_path, "-c:a", "libvorbis", ogg_path]
            print("실행 커맨드:", " ".join(command))

            try:
                result = subprocess.run(command, capture_output=True, text=True)
                print(result.stderr)  # 🔥 여기에서 문제 확인 가능

                if os.path.exists(ogg_path):
                    print(f"🔁 ogg 생성 완료: {filename}")
                else:
                    print(f"❌ ogg 파일 생성 실패: {filename}")
            except Exception as e:
                print(f"❌ ffmpeg 실행 오류: {e}")

    print(f"📁 최종 출력 폴더: {output_folder}")
    return output_folder


In [3]:
ogg_dir = "D:/Download/birdclef-2025/train_audio/21038"

# 1. ogg → wav
wav_dir = convert_ogg_to_wav(ogg_dir)

# 2. 필터링
filtered_wav_dir = filter_wav(wav_dir, lowcut=2000, highcut=5000, order=5)

# 3. wav → ogg (ffmpeg 사용)
ogg_result_dir = convert_wav_to_ogg_ffmpeg(filtered_wav_dir)

🎧 ogg → wav 성공: iNat297879.ogg
🎧 ogg → wav 성공: iNat65519.ogg
✅ 필터링 완료: iNat297879.wav
✅ 필터링 완료: iNat65519.wav


PermissionError: [Errno 13] Permission denied: 'species_filter_log.csv'